# Перевірка міграції

## Допоміжна документація
- [Introduction to the Python Driver for Oracle Database](https://python-oracledb.readthedocs.io/en/latest/user_guide/introduction.html#introduction-to-the-python-driver-for-oracle-database)
- [Python python-oracledb Driver](https://oracle.github.io/python-oracledb/)
- [Python: Read Data from Oracle Database](https://kontext.tech/article/1019/python-read-data-from-oracle-database)

## Підключення до БД через "тонкий" клієнт ORACLE та виборка одного запису з його виводом через "print"
- Очікується, що вибиирається тільки один запис [5.1.1. Fetch Methods](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#fetch-methods)
- Прив'язка зміних запиту виконується через **змінні прив'язки**

In [2]:
    import oracledb
    import getpass
    
    username = "CUSTDOC"
    userpwd = getpass.getpass("Enter password: ")

    host="localhost"
    port="1521"
    service_name="XEPDB1"
    dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
   

Enter password:  ········


## Очистка таблиці CUSTDOC.CUST$DOCS$URLS


In [22]:
import oracledb
import os

try:
    connection = oracledb.connect(dsn)
    cursor = connection.cursor()
    sqld="""DELETE FROM CUSTDOC.CUST$DOCS$URLS"""
    cursor.execute(sqld) 
    connection.commit();
except oracledb.Error as e:
    # Обробка помилок Oracle
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
except Exception as error:
    print(f" Виникла неочікувана помилка: {type(error).__name__}")
    print(f"Деталі помилки: {error}")
    print(sys.exc_info())    
finally:
    # 6. Закриття курсора та з'єднання
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")


Курсор закрито.
З'єднання з БД закрито.


## Очистка таблиці за умовою (по FLID)

In [9]:
import oracledb
import os

try:
    connection = oracledb.connect(dsn)
    cursor = connection.cursor()
    sqld="""DELETE FROM CUSTDOC.CUST$DOCS$URLS WHERE IDDOC=:A_IDDOC"""
    cursor.execute(sqld, { "A_IDDOC":  2563 }) 
    connection.commit();
except oracledb.Error as e:
    # Обробка помилок Oracle
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
except Exception as error:
    print(f" Виникла неочікувана помилка: {type(error).__name__}")
    print(f"Деталі помилки: {error}")
    print(sys.exc_info())    
finally:
    # 6. Закриття курсора та з'єднання
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")


Курсор закрито.
З'єднання з БД закрито.


#### Перегляд тестових даних

```text
  IDFL         INTEGER NOT NULL,
  IDDOC        INTEGER ,
  FILE_NAME    VARCHAR2(120),
  CONTAINER_NAME  VARCHAR2(120),
  CONTENT_TYPE VARCHAR2(80),
  FILE_SIZE    INTEGER,
  FILE_URL     VARCHAR2(240),
  FILE_DESC    VARCHAR2(120),
  IUSRNM       VARCHAR2(30),
  IDT          DATE,
  MUSRNM       VARCHAR2(30),
  MDT          DATE
  ```

In [116]:
import pandas as pd
from IPython.display import display

sql_query = """
    SELECT A.IDFL, A.IDDOC, A.FILE_NAME, A.CONTENT_TYPE, A.FILE_SIZE, A.FILE_URL, A.IDT, A.IUSRNM
    FROM CUSTDOC.CUST$DOCS$URLS  A
 """

connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення

try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
        print(f"Fetched rows={len(rows)}")
    else:
        print("Запит не повернув жодного рядка.")
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

,IDFL,IDDOC,FILE_NAME,CONTENT_TYPE,FILE_SIZE,FILE_URL,IDT,IUSRNM
0,3618,3617,CU_000000002_IMG_20190512_123820.jpg,image/jpeg,1444803,https://custstrgsrvc.blob.core.windows.net/cus...,None,None
1,3621,3619,CU_000000003_IMG_20190512_124107.jpg,image/jpeg,1173695,https://custstrgsrvc.blob.core.windows.net/cus...,None,None
2,3620,3619,CU_000000003_IMG_20190512_123820.jpg,image/jpeg,1444803,https://custstrgsrvc.blob.core.windows.net/cus...,None,None
3,3624,3622,CU_000000003_IMG_20190512_124206.jpg,image/jpeg,1385367,https://custstrgsrvc.blob.core.windows.net/cus...,None,None
4,3623,3622,CU_000000003_IMG_20190515_135249.jpg,image/jpeg,1630310,https://custstrgsrvc.blob.core.windows.net/cus...,None,None
5,3626,3625,CU_000000004_IMG_20190512_123820.jpg,image/jpeg,1444803,https://custstrgsrvc.blob.core.windows.net/cus...,None,None
6,3629,3627,CU_000000005_1000001462.jpg,image/jpeg,5892222,https://custstrgsrvc.blob.core.windows.net/cus...,None,None
7,3628,3627,CU_000000005_1000001465.jpg,image/jpeg,1560713,https://custstrgsrvc.blob.core.windows.net/cus...,None,None
8,3631,3630,CU_000000006_IMG_20190512_123820.jpg,image/jpeg,1444803,https://custstrgsrvc.blob.core.windows.net/cus...,None,None
9,3633,3632,CU_000000007_IMG_20190511_152919.jpg,image/jpeg,1622516,https://custstrgsrvc.blob.core.windows.net/cus...,None,None


Fetched rows=16

Курсор закрито.
З'єднання з БД закрито.


# Пошук не мігрованих ще записів в БД

In [31]:
import pandas as pd
from IPython.display import display

sql_query = """
    SELECT A.CUSTID, A.IDDOC, A.ISACRUAL FROM CUSTDOC.CUST$DOCS  A 
        WHERE NOT EXISTS( SELECT 1 FROM CUSTDOC.CUST$DOCS$URLS B WHERE  B.IDDOC=A.IDDOC) 
 """
connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення

try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
        print(f"Fetched rows={len(rows)}")
    else:
        print("Запит не повернув жодного рядка.")
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

,CUSTID,IDDOC,ISACRUAL
0,000000058,2545,N
1,000000084,2614,Y
2,000000085,2616,Y
3,000000086,2619,Y
4,000000087,2621,Y
...,...,...,...
96,000000053,2530,Y
97,000000054,2532,Y
98,000000055,2534,Y
99,000000056,2537,Y


Fetched rows=101

Курсор закрито.
З'єднання з БД закрито.


# Агреговане відображення міграції

In [120]:
import pandas as pd
from IPython.display import display
sql_query = """
    SELECT 'NUMBER OF DOCUMENTS' as METRICS, COUNT(*) AS RECORD_COUNT 
    FROM CUSTDOC.CUST$DOCS  A
    UNION ALL
    SELECT 'JUST SENT NOT CONFIRMED' as METRICS, COUNT(*) AS RECCNT  
        FROM CUSTDOC.CUST$DOCS  A
        WHERE NOT EXISTS( SELECT 1 FROM CUSTDOC.CUST$DOCS$URLS B WHERE  B.IDDOC=A.IDDOC) 
        AND A.ISACRUAL='N'
    UNION ALL
    SELECT 'MIGRATED AND CONFIRMED' as METRICS, COUNT(*) AS RECCNT  
        FROM CUSTDOC.CUST$DOCS  A
        WHERE  EXISTS( SELECT 1 FROM CUSTDOC.CUST$DOCS$URLS B WHERE  B.IDDOC=A.IDDOC) 
        AND A.ISACRUAL='N'        
    UNION ALL
    SELECT 'NOT MIGRATED' as METRICS, COUNT(*) AS RECCNT  
        FROM CUSTDOC.CUST$DOCS  A
        WHERE NOT EXISTS( SELECT 1 FROM CUSTDOC.CUST$DOCS$URLS B WHERE  B.IDDOC=A.IDDOC) 
        AND A.ISACRUAL='Y'
 """
connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
        print(f"Fetched rows={len(rows)}")
    else:
        print("Запит не повернув жодного рядка.")
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

,METRICS,RECORD_COUNT
0,NUMBER OF DOCUMENTS,131
1,JUST SENT NOT CONFIRMED,1
2,MIGRATED AND CONFIRMED,76
3,NOT MIGRATED,54


Fetched rows=4

Курсор закрито.
З'єднання з БД закрито.


```mermaid
graph TD;
    A-->B;
    A-->C;
    B-->D;
    C-->D;
```

```mermaid
erDiagram
    CUSTDOC_CUST ||--o{ CUSTDOC_CUST_DOCS : has
    CUSTDOC_CUST_DOCS ||--o{ CUSTDOC_CUST_DOCS_ATTACH : includes

    CUSTDOC_CUST {
        varchar CustID PK
        varchar CustomerRecords
    }

    CUSTDOC_CUST_DOCS {
        varchar DocID PK
        varchar CustID FK "Посилається на CUSTDOC.CUST$CUST"
        varchar CustomerDocuments
    }

    CUSTDOC_CUST_DOCS_ATTACH {
        varchar AttachID PK
        varchar DocID FK "Посилається на CUSTDOC.CUST$DOCS"
        varchar AttachedFiles
    }
```    
    